In [1]:
pip install -U pygame --user

Requirement already up-to-date: pygame in c:\users\goodr\appdata\roaming\python\python38\site-packages (2.0.1)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pygame, sys
from pygame.locals import *
import io
import math
import pygame.math as pyMath
from urllib.request import urlopen
from random import randint

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
#conda install -c cogsci pygame

In [4]:
pygame.init()

(7, 0)

In [5]:
screen = pygame.display.set_mode((800, 600))

In [6]:
running = True
isAlive = True

In [7]:
#Sprites
icon = pygame.image.load('./duck.png')

In [8]:
#Backgrounds
load_bg = pygame.image.load("../bg.jpg")
blackHole_img = pygame.image.load("./blackHole.jpg")
blackHole_bg = pygame.transform.scale(blackHole_img, (800, 600))
bg = pygame.transform.scale(load_bg, (800, 600))
level0 = pygame.image.load("./level0.jpg")

In [9]:
#Global Variables
pygame.display.set_caption("Qwack")
meme = pygame.image.load('./qwackMeme.jpg')
pygame.display.set_icon(meme)

fontTitle = pygame.font.SysFont("Comic", 125)
fontSubHeader = pygame.font.SysFont(None, 50)
text = pygame.font.SysFont(None, 25)

mainClock = pygame.time.Clock()

click = False
themeColor = (255,255,0)


In [10]:
def draw_text(text, font, color, surface, x, y):
    textObj = font.render(text,1,color)
    textRect = textObj.get_rect()
    textRect.topleft = (x,y)
    surface.blit(textObj, textRect)
    return textRect

In [11]:
#icon
global iconX, iconY, velocity
iconX = randint(10, 768)
iconY = randint(10, 568)
velocity = [2.5,1.5]

In [12]:
def Icon(sprite, x,y):
    screen.blit(sprite, (x, y))
    rect = pygame.Rect(x, y, 32, 32)
    return rect

In [13]:
def Distance(x2, y2):
    hole = pyMath.Vector2(400, 300)
    dist = hole.distance_to(pyMath.Vector2(x2, y2))
    return dist

In [14]:
def Magnitude(x, y):
    return (x ** 2 + y ** 2) ** 0.5

In [15]:
def changeGrav(duckHole, grav):
    newDuckHole = pyMath.Vector2(duckHole.x/grav, duckHole.y/grav)
    return pyMath.Vector2(newDuckHole.x+velocity[0], newDuckHole.y+velocity[1])

In [16]:
def main_menu(iconX, iconY, isAlive):
    blackHole_rect = pygame.Rect(382, 287, 32, 32)  # blackHole hitbox
    duck_button = pygame.Rect(0,0,32,32) # duck hitbox
    hole = pyMath.Vector2(400, 300) #blackHole x, y position in vector
    blackHole = False # if True, blackHole code runs
    previous_coord = pyMath.Vector2(0,0) # storing inital duck coordinates before movement 
    dist = 0 # distance value init
    
    while True:
        
        # Setup
        previous_coord.x = iconX  # setting x and y values
        previous_coord.y = iconY

        if blackHole == False: 
            screen.blit(bg, (0,0)) # regular background image render
        else: 
            screen.blit(blackHole_bg, (0,0)) # black hole background image render
        mx, my = pygame.mouse.get_pos()   # get mouse position for interaction    
            
        # blackHole gravity    
                
        dist = Distance(iconX, iconY) # call distance function to calculate black hole gravity 
            
        # initial velocity (no black hole)
        if iconX >= 785:
            velocity[0] = -velocity[0]
        if iconX <= 0:
            velocity[0] = -velocity[0]
        if iconY > 585:
            velocity[1] = -velocity[1]
        if iconY < 0:
            velocity[1] = -velocity[1] 
                
        
        if blackHole: # velocity with black hole
            duck_to_hole = pyMath.Vector2((400-previous_coord.x), (300-previous_coord.y)) # vector from initial duck position to black hole
            resultant = pyMath.Vector2(duck_to_hole.x + velocity[0], duck_to_hole.y + velocity[1]) # resultant vector from duck to hole and velocity vectors

            # calculating varying gravity from duck Distance to black hole
            if (dist/175) > .99: # gravity influence starting point 
                iconX += velocity[0]
                iconY += velocity[1]
            elif (dist/175) >= .75 and (dist/175) <= .99: # first variation 
                resultant = changeGrav(duck_to_hole, 50) # call changeGrav function to add the gravity influence to duck vector
                if abs(resultant.x) <= 1 and abs(resultant.y) <= 1: 
                    iconX += velocity[0]-resultant.x
                    iconY += velocity[1]-resultant.y
                else:
                    iconX += resultant.x
                    iconY += resultant.y
            elif (dist/175) >= .50 and (dist/175) < .75: # second variation 
                resultant = changeGrav(duck_to_hole, 40) # gravity is a little stronger; smaller the number, the stronger the gravity
                if abs(resultant.x) <= 1 and abs(resultant.y) <= 1:
                    iconX += velocity[0]-resultant.x
                    iconY += velocity[1]-resultant.y
                else:
                    iconX += resultant.x
                    iconY += resultant.y
            elif (dist/175) >= .25 and (dist/175) < .5: # third variation 
                resultant = changeGrav(duck_to_hole, 20)
                if abs(resultant.x) <= 1 and abs(resultant.y) <= 1:
                    iconX += velocity[0]-resultant.x
                    iconY += velocity[1]-resultant.y
                else:
                    iconX += resultant.x
                    iconY += resultant.y
            elif (dist/175) < .25:
                resultant = changeGrav(duck_to_hole, 1) # final variation, pulls duck in all the way (event horizion)
                if abs(resultant.x) <= 1 and abs(resultant.y) <= 1:
                    iconX += velocity[0]-resultant.x
                    iconY += velocity[1]-resultant.y
                else:
                    iconX += resultant.x
                    iconY += resultant.y
            if duck_button.colliderect(blackHole_rect): # collision detection between the duck and black hole to trigger duck vanishing
                isAlive = False
                blackHole = False
                
        else:  # if no black hole
            iconX += velocity[0]
            iconY += velocity[1]
        
#         duck_to_hole_mag = Magnitude(duck_to_hole.x, duck_to_hole.y)
#         prev_to_cur_mag = Magnitude(velocity[0],velocity[1])
        
        # determines whether to show duck or not
        if isAlive:
            duck_button = Icon(icon, iconX, iconY)
        
        # Text
        home_button = draw_text("Qwack", fontTitle, themeColor, screen, 260, 15)
        start_game_button = draw_text("Start Game", fontSubHeader, themeColor, screen, 75, 150)
        controls_button = draw_text("Controls", fontSubHeader, themeColor, screen, 75, 225)
        credits_button = draw_text("Credits", fontSubHeader, themeColor, screen, 75, 300)
        
        # Event Handling
        click = False
        for event in pygame.event.get():
            if event.type == QUIT: #if user wants to exit game
                pygame.quit()
                sys.exit()
            if event.type == KEYDOWN: 
                if event.key == K_ESCAPE: #when user presses esc it leaves game
                    pygame.quit()
                    sys.exit()
            if event.type == MOUSEBUTTONDOWN:
                if event.button == 1: # checking for clicks
                    click = True
                    #print(mx, my)
        
        # Checking for user clicks on buttons
        if home_button.collidepoint((mx, my)): # if user clicks on 'Qwack' title, returns to menu page
            if click:
                main_menu(iconX, iconY, isAlive)            
        if start_game_button.collidepoint((mx, my)): # if user clicks on 'Level Select', goes to level select page
            if click:
                level_select(iconX, iconY, isAlive)
        if controls_button.collidepoint((mx, my)): # if user clicks on 'Controls', goes to controls page
            if click:
                Controls(iconX, iconY, isAlive)
        if credits_button.collidepoint((mx, my)): # if user clicks on 'Credits', goes to credits page
            if click:
                Credits(iconX, iconY, isAlive)
        if duck_button.collidepoint((mx, my)): # if user clicks on floating duck icon, black hole appears
            if click:
                blackHole = True
                
        
        pygame.display.update() # update the game with changes from this loop
        mainClock.tick(60) # FPS

In [17]:
def level_select(iconX, iconY, isAlive):
    running = True
    while running:
        
        #Initialize
        screen.fill((0,0,0))
        screen.blit(bg, (0,0))
        mx, my = pygame.mouse.get_pos()
        
        #icon bouncing in background
        if iconX >= 785:
            velocity[0] = -velocity[0]
        if iconX <= 0:
            velocity[0] = -velocity[0]
        if iconY > 585:
            velocity[1] = -velocity[1]
        if iconY < 0:
            velocity[1] = -velocity[1] 
            
            
        iconX += velocity[0]
        iconY += velocity[1]
        if isAlive:
            duck_button = Icon(icon, iconX, iconY)
        
        home_button = draw_text("Qwack", fontTitle, themeColor, screen, 262, 15)
        draw_text("Level Select", fontSubHeader, (255, 255, 255), screen, 307, 110)
        draw_text("___________", fontSubHeader, (255, 255, 255), screen, 307.25, 112)
        esc_button = draw_text("Press Esc To Return", text, themeColor, screen, 10, 15)
        buttonLevel_0 = draw_text("Level 0: Tutorial", text, (255, 255, 255), screen, 75, 175)
        buttonLevel_1 = draw_text("Level 1: Entanglement", text, (255, 255, 255), screen, 75, 225)
        buttonLevel_2 = draw_text("Level 2:", text, (255, 255, 255), screen, 75, 275)
        buttonLevel_3 = draw_text("Level 3:", text, (255, 255, 255), screen, 75, 325)
        buttonLevel_4 = draw_text("Level 4:", text, (255, 255, 255), screen, 75, 375)
        buttonLevel_5 = draw_text("Level 5:", text, (255, 255, 255), screen, 75, 425)
        buttonLevel_6 = draw_text("Level 6:", text, (255, 255, 255), screen, 75, 475)
        
        #player
        playerX = 370
        playerY = 490
        xChange = 0
        yChange = 0
        
        
            
        click = False
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            if event.type == KEYDOWN:
                if event.key == K_ESCAPE:
                    running = False
            if event.type == MOUSEBUTTONDOWN:
                if event.button == 1:
                    click = True
                    
        if home_button.collidepoint((mx, my)) or esc_button.collidepoint((mx, my)):
            if click:
                main_menu(iconX, iconY, isAlive) 
        if buttonLevel_0.collidepoint((mx, my)):
            if click:
                level_0(playerX, playerY, xChange, yChange)
        if buttonLevel_1.collidepoint((mx, my)):
            if click:
                pass
        if buttonLevel_2.collidepoint((mx, my)):
            if click:
                pass
        pygame.display.update()
        mainClock.tick(60)

In [18]:
def Controls(iconX, iconY, isAlive):
    running = True
    while running:
        screen.fill((0,0,0))
        screen.blit(bg, (0,0))
        mx, my = pygame.mouse.get_pos()
        
        #icon bouncing in background
        if iconX >= 785:
            velocity[0] = -velocity[0]
        if iconX <= 0:
            velocity[0] = -velocity[0]
        if iconY > 585:
            velocity[1] = -velocity[1]
        if iconY < 0:
            velocity[1] = -velocity[1] 
            
            
        iconX += velocity[0]
        iconY += velocity[1]
        if isAlive:
            duck_button = Icon(icon, iconX, iconY)
        
        home_button = draw_text("Qwack", fontTitle, themeColor, screen, 262, 15)
        esc_button = draw_text("Press Esc To Return", text, themeColor, screen, 10, 15)
        draw_text("Controls", fontSubHeader, (255, 255, 255), screen, 335, 110)
        draw_text("________", fontSubHeader, (255, 255, 255), screen, 336, 112)
        
        click = False
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            if event.type == KEYDOWN:
                if event.key == K_ESCAPE:
                    running = False
            if event.type == MOUSEBUTTONDOWN:
                if event.button == 1:
                    click = True        
        if home_button.collidepoint((mx, my)) or esc_button.collidepoint((mx, my)):
            if click:
                main_menu(iconX, iconY, isAlive) 
                
        pygame.display.update()
        mainClock.tick(60) 

In [19]:
def Credits(iconX, iconY, isAlive):
    running = True
    while running:
        screen.fill((0,0,0))
        screen.blit(bg, (0,0))
        mx, my = pygame.mouse.get_pos()
        
        #icon bouncing in background
        if iconX >= 785:
            velocity[0] = -velocity[0]
        if iconX <= 0:
            velocity[0] = -velocity[0]
        if iconY > 585:
            velocity[1] = -velocity[1]
        if iconY < 0:
            velocity[1] = -velocity[1] 
            
            
        iconX += velocity[0]
        iconY += velocity[1]
        if isAlive:
            duck_button = Icon(icon, iconX, iconY)
        
        home_button = draw_text("Qwack", fontTitle, themeColor, screen, 262, 15)
        esc_button = draw_text("Press Esc To Return", text, themeColor, screen, 10, 15)
        draw_text("Credits", fontSubHeader, (255, 255, 255), screen, 312, 110)
        draw_text("________", fontSubHeader, (255, 255, 255), screen, 312.25, 112)
        
        click = False
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            if event.type == KEYDOWN:
                if event.key == K_ESCAPE:
                    running = False
            if event.type == MOUSEBUTTONDOWN:
                if event.button == 1:
                    click = True        
        if home_button.collidepoint((mx, my)) or esc_button.collidepoint((mx, my)):
            if click:
                main_menu(iconX, iconY, isAlive) 
                    
        pygame.display.update()
        mainClock.tick(60) 

In [20]:
def level_0(x, y, xChange, yChange):
    running = True
    jump = False
    fall = False
    inst = True
    i = 0
    bg_movement = 0
    width = 800
    #print(y)
    while running:
        
        screen.fill((0,0,0))
        screen.blit(level0, (bg_movement,0))
        screen.blit(level0, (width+bg_movement, 0))
        
        if bg_movement == -width:
            screen.blit(level0, (width+bg_movement, 0))
            bg_movement = 0
        bg_movement -= 1
        # Setup
        mx, my = pygame.mouse.get_pos()
        instructions = ["Hello! Welcome to Qwack!", 
                       "This is a game about exploring and learning Quantum Computing Concepts",
                       "Meet Dr. Qwack! He is stuck in a quantum realm and needs your help to escape!",
                       "Use the LEFT ARROW and RIGHT ARROW keys to move",
                       "Use SPACE to Jump",
                       "If at any point you want to abandon Dr. Qwack, press ESC"]
        #Instructions
        if i == 0:
            if inst:
                draw_text("Use the SPACE Button to continue reading...", fontSubHeader, (255, 255, 255), screen, 50, 200)
        text_rect = draw_text(instructions[i], text, (255, 255, 255), screen, 120, 400)
    
        #print(text_rect.top)
        #Event Handling
        click = False
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            if event.type == KEYDOWN:
                if event.key == K_ESCAPE:
                    running = False
                # movement
                if event.key == pygame.K_LEFT:
                    xChange = -5
                if event.key == pygame.K_RIGHT:
                    xChange = 5
                if event.key == pygame.K_UP and jump is False:
                    jump = True
                    yChange = 15
                if event.key == pygame.K_SPACE and i < len(instructions)-1:
                    i += 1
                elif event.key == pygame.K_SPACE and i == len(instructions)-2:
                    print(here)
                    inst = False
                    i = 0
            if event.type == pygame.KEYUP:
                if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                    xChange = 0
            if event.type == MOUSEBUTTONDOWN:
                if event.button == 1:
                    click = True
                    # print(mx, my)
                    
        # Set movement Values
        
        
        x += xChange
        if x <= 0:
            x = 0
        elif x >= 780:
            x = 780
            
        if jump and not fall:
            y -= yChange
            yChange -= 1
            
            if yChange < -15:
                jump = False
                yChange = 0
        if y == 370 and x >= text_rect.left and x <= text_rect.right:
            jump = False
            y = 370
        elif y == 370 and (x <= text_rect.left or x >= text_rect.right):
            fall = True
            yChange = 1
            #print('here')
            
        if fall:
            y += yChange
            yChange += 1
            if yChange > 15:
                fall = False
                yChange = 0
        
        if y > 490:
            y = 490
            fall = False
        #Display
        Icon(icon, x, y)
        pygame.display.update()
        mainClock.tick(60) 

In [21]:
# Game Loop
main_menu(iconX, iconY, isAlive)


SystemExit: 

C:\Users\goodr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
''' Formatting changes to make:

    Menu
    -Start
        -Level Select
        -Level 0 (story intro, tutorial)
    -Controls
    -Credits
    
    add note about "Hit Esc to go back"
    
    Color scheme??
'''